<H1>PyCity School Analysis</H1>

As a whole...

In [195]:
#import dependencies
import pandas as pd

#file path variables
schools = "../resources/schools_complete.csv"
students = "../resources/students_complete.csv"

#read csvs
schools_df = pd.read_csv(schools)
students_df = pd.read_csv(students)

#merging the school and student data
merge_df = pd.merge(schools_df,students_df,how="left",on=["school_name","school_name"])
merge_df.head()

,School ID,school_name,type,size,budget,Student ID,student_name,gender,grade,reading_score,math_score
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,66,79
1,0,Huang High School,District,2917,1910635,1,Victor Smith,M,12th,94,61
2,0,Huang High School,District,2917,1910635,2,Kevin Rodriguez,M,12th,90,60
3,0,Huang High School,District,2917,1910635,3,Dr. Richard Scott,M,12th,67,58
4,0,Huang High School,District,2917,1910635,4,Bonnie Ray,F,9th,97,84


<H1>District Summary</H1>

In [196]:
#group the data by school name
grouped_data = merge_df.groupby("school_name")

In [197]:
#Total number of unique schools
total_schools = len(grouped_data["school_name"].unique())

In [198]:
#Total students
total_students = grouped_data["size"].mean().sum()

In [199]:
#Total budget
total_budget = grouped_data["budget"].mean().sum()

In [200]:
#Average math score
math_avg = merge_df["math_score"].mean()

#Average reading score
reading_avg = merge_df["reading_score"].mean()

In [201]:
# Use the following to calculate the percentage of students who passed math (math scores greather than or equal to 70)
math_pass_count = merge_df[(merge_df["math_score"] >= 70)].count()["student_name"]
math_pass_rate = math_pass_count / float(total_students) * 100

#% passing reading (the percentage of students who passed reading)
reading_pass_count = merge_df[(merge_df["reading_score"] >= 70)].count()["student_name"]
reading_pass_rate = reading_pass_count / float(total_students) * 100


#% overall passing (the percentage of students who passed math AND reading)
passing_both_count = merge_df[
    (merge_df["math_score"] >= 70) & (merge_df["reading_score"] >= 70)
].count()["student_name"]
overall_pass_rate = passing_both_count /  float(total_students) * 100
overall_pass_rate

65.17232575950983

In [202]:
district_summary = pd.DataFrame({
    "Total_Schools": [total_schools], 
    "Total_Students": [total_students],
    "Total_Budget": [total_budget],
    "Total_Budget": [total_budget],
    "Average_Math_Score": [math_avg], 
    "Average_Reading_Score": [reading_avg], 
    "%_Passing_Math": [math_pass_rate],
    "%_Passing_Reading": [reading_pass_rate],
    "%_Overall_Passing": [overall_pass_rate],
})
pd.DataFrame(district_summary)

# Formatting
district_summary["Total_Students"] = district_summary["Total_Students"].map("{:,}".format)
district_summary["Total_Budget"] = district_summary["Total_Budget"].map("${:,.2f}".format)

# Display the DataFrame
district_summary

,Total_Schools,Total_Students,Total_Budget,Average_Math_Score,Average_Reading_Score,%_Passing_Math,%_Passing_Reading,%_Overall_Passing
0,15,"39,170.0","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


<H1>School Summary</H1>

In [203]:
# Use the code provided to select the school type
school_types = schools_df.set_index(["school_name"])["type"]

In [204]:
# Calculate the total student count
per_school_counts = grouped_data.mean()["size"]

In [205]:
# Calculate the total school budget and per capita spending
per_school_budget = grouped_data.mean()["budget"]
per_school_capita = per_school_budget / per_school_counts

In [206]:
# Calculate the average test scores
per_school_math = grouped_data.mean("math_score")
per_school_reading = grouped_data.mean("reading_score")

In [207]:
# Calculate the number of schools with math scores of 70 or higher
school_passing_math = merge_df[
    (merge_df["math_score"] >= 70)]

In [208]:
# Calculate the number of schools with reading scores of 70 or higher
school_passing_reading = merge_df[
    (merge_df["reading_score"] >= 70)]

In [209]:
# Use the provided code to calculate the schools that passed both math and reading with scores of 70 or higher
passing_math_and_reading = merge_df[
    (merge_df["reading_score"] >= 70) & (merge_df["math_score"] >= 70)
]

In [210]:
# Use the provided code to calculate the passing rates
per_school_passing_math = school_passing_math.groupby(["school_name"]).count()["student_name"] / per_school_counts * 100
per_school_passing_reading = school_passing_reading.groupby(["school_name"]).count()["student_name"] / per_school_counts * 100
overall_passing_rate = passing_math_and_reading.groupby(["school_name"]).count()["student_name"] / per_school_counts * 100

In [218]:
# Create a DataFrame called `per_school_summary` with columns for the calculations above.
per_school_summary = pd.DataFrame({
    "School_Types": school_types, 
    "Student_Count": per_school_counts,
    "Total_School_Budget": per_school_budget,
    "Per_Capita_Budget": per_school_capita,
    "Average_Math_Score": per_school_math, 
    "Average_Reading_Score": per_school_reading, 
    "%_Passing_Math": per_school_passing_math,
    "%_Passing_Reading": per_school_passing_reading,
    "%_Overall_Passing": overall_passing_rate})

# # Formatting
# per_school_summary["Total_School_Budget"] = per_school_summary["Total_School_Budget"].map("${:,.2f}".format)
# per_school_summary["Per_Capita_Budget"] = per_school_summary["Per_Capita_Budget"].map("${:,.2f}".format)

# Display the DataFrame
per_school_summary

ValueError: Data must be 1-dimensional